In [1]:
import pygame
import numpy as np
import os, sys

pygame 2.5.2 (SDL 2.28.2, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
WIDTH = 640
HEIGHT = 1200
LINE_WIDTH = 8
WIN_LINE_WIDTH = 15

BOARD_ROWS = 4
BOARD_COLS = 4
BOARD_FLOORS = 4

SQUARE_SIZE = 50

CIRCLE_RADIUS = 20
CIRCLE_WIDTH = 4

CROSS_WIDTH = 8

SPACE = 8
GAP = 80
TEXT_LENGTH = 200

RED = (255, 0, 0)
BG_COLOR = (20, 200, 160)
LINE_COLOR = (23, 145, 135)
CIRCLE_COLOR = (239, 231, 200)
CROSS_COLOR = (66, 66, 66)

In [3]:
class TicTacToe3D:
    def __init__(self, headless=True):
        self.board = np.zeros((4, 4, 4), dtype=int)
        self.heights = np.zeros((4, 4), dtype=int)
        self.screen = None
        self.all_wins = TicTacToe3D.calculateAllWin()

        if not headless:
            pygame.init()

            self.screen = pygame.display.set_mode( (WIDTH, HEIGHT) )
            pygame.display.set_caption( '3D TIC TAC TOE' )
            self.screen.fill( BG_COLOR )

            self.clock = pygame.time.Clock()
            self.fps = 60

    def reset(self):
        self.board = np.zeros((4, 4, 4), dtype=int)
        self.heights = np.zeros((4, 4), dtype=int)

        if self.screen is not None:
            self.screen.fill( BG_COLOR )
        return self.board, self.getPossibleMove()

    def calculateAllWin():
        oneD = [(1,0,0), (0,1,0), (0,0,1)]
        twoD = [(1,1,0), (1,0,1), (0,1,1), (1,-1,0), (1,0,-1), (0,1,-1)]
        threeD = [(1,1,1), (1,1,-1), (1,-1,1), (-1,1,1)]
        win_masks = oneD + twoD + threeD

        all_wins = []
        tmp = np.zeros((4, 4, 4), dtype=int)
        for i in range(4):
            for j in range(4):
                for k in range(4):
                    for dx, dy, dz in win_masks:
                        for l in range(4):
                            if i + dx * l < 0 or i + dx * l >= 4 or j + dy * l < 0 or j + dy * l >= 4 or k + dz * l < 0 or k + dz * l >= 4:
                                break
                            tmp[i + dx * l, j + dy * l, k + dz * l] = 1
                        else:
                            all_wins.append(tmp.copy())
                        tmp.fill(0)

        return all_wins

    def evalMove(self, row, col, player):
        '''
        looped every possible win
        Count1 = 13 break | if win
                0 | nothing

        looped every possible win
        Count2 = 6 | block
                0 | nothing

        looped every possible win
        Count3 += 1 | win con
                0 | nothing

        looped every possible win
        Count4 = -13 | create chance for opposition
                0 | nothing

        looped every possible win
        Count5 = 0 | miss && Count1 = 13
                -13 | miss

        Reward / Steps = Count1 + Count2 + Count3 + Count4
        '''
        
        def createLosingChance(future_board1, player):
            for r in range(4):
                for c in range(4):
                    future_board2 = future_board1.copy()
                    if self.heights[r,c] >= 4:
                        continue
                    future_board2[self.heights[r,c], r, c] = -player
                    if self.check(board=future_board2) == -player:
                        return True

        count1 = 0
        if self.check() == player:
            count1 = 13
        
        count2 = 0
        count3 = 0
        for win in self.all_wins:
            line = win * self.board
            #check if row col is in the line
            height = self.heights[row, col]
            if line[height - 1, row, col] != player:
                continue
            if np.count_nonzero(line == -player) == 3 and np.count_nonzero(line == player) == 1:
                count2 = 6
            if np.count_nonzero(line == player) == 3 and np.count_nonzero(line == -player) == 0:
                count3 += 1

        if count1 != 13:
            count4 = 0
            count5 = 0
            previous_board = self.board.copy()
            previous_board[self.heights[row, col] - 1, row, col] = 0
            previous_heights = self.heights.copy()
            previous_heights[row, col] -= 1
            for r in range(4):
                for c in range(4):
                    future_board = previous_board.copy()
                    if previous_heights[r,c] >= 4:
                        continue
                    future_board[previous_heights[r,c], r, c] = player
                    if self.check(board=future_board) == player:
                        count5 = -13
                    if createLosingChance(future_board, player):
                        count4 = -13
        else:
            count4 = 0
            count5 = 0

        return count1 + count2 + count3 + count4 + count5

    def move(self, row, col, player):
        if not (0 <= row < 4 and 0 <= col < 4):
            return False, self.board, self.getPossibleMove(), -99, self.check() != 0

        floor = self.heights[row, col]
        if floor >= 4:
            return False, self.board, self.getPossibleMove(), -99, self.check() != 0
        if self.board[floor, row, col] != 0:
            return False, self.board, self.getPossibleMove(), -99, self.check() != 0

        self.board[floor, row, col] = player
        self.heights[row, col] += 1

        return True, self.board, self.getPossibleMove(), self.evalMove(row, col, player), self.check() != 0

    def getPossibleMove(self):
        possible_moves = []
        for row in range(4):
            for col in range(4):
                if self.heights[row, col] < 4:
                    possible_moves.append((row, col))
        return possible_moves

    def check(self, board=None):
        # check if any player has connected 4 in a row
        if board is None:
            board = self.board
        for i in range(4):
            for j in range(4):
                for k in range(4):
                    if board[i, j, k] == 0:
                        continue
                    oneD = [(1,0,0), (0,1,0), (0,0,1)]
                    twoD = [(1,1,0), (1,0,1), (0,1,1), (1,-1,0), (1,0,-1), (0,1,-1)]
                    threeD = [(1,1,1), (1,1,-1), (1,-1,1), (-1,1,1)]
                    for dx, dy, dz in oneD + twoD + threeD:
                        for l in range(1, 4):
                            if i + dx * l < 0 or i + dx * l >= 4 or j + dy * l < 0 or j + dy * l >= 4 or k + dz * l < 0 or k + dz * l >= 4 or board[i + dx * l, j + dy * l, k + dz * l] != board[i, j, k]:
                                break
                        else:
                            return board[i, j, k]
        return 0
    
    def loadState(self, board):
        self.board = board.copy()
        # self.board = board.detach().clone().to(device)
        # last_non_zero = np.argmax(board[::-1] != 0, axis=0)
        # self.heights = np.where(last_non_zero == 0, 0, self.board.shape[0] - last_non_zero)
        self.heights = np.sum(board != 0, axis=0)
        return self
                        
    def draw_lines(self):
        if self.screen is None:
            return
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, GAP + 1 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, GAP + 1 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, GAP + 2 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, GAP + 2 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, GAP + 3 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, GAP + 3 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + SQUARE_SIZE, GAP), (GAP + TEXT_LENGTH + GAP + SQUARE_SIZE, GAP + 4 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + 2 * SQUARE_SIZE, GAP), (GAP + TEXT_LENGTH + GAP + 2 * SQUARE_SIZE, GAP + 4 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + 3 * SQUARE_SIZE, GAP), (GAP + TEXT_LENGTH + GAP + 3 * SQUARE_SIZE, GAP + 4 * SQUARE_SIZE), LINE_WIDTH )
        
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, 2 * GAP + 5 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, 2 * GAP + 5 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, 2 * GAP + 6 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, 2 * GAP + 6 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, 2 * GAP + 7 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, 2 * GAP + 7 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + 1 * SQUARE_SIZE, 2 * GAP + 4 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 1 * SQUARE_SIZE, 2 * GAP + 8 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + 2 * SQUARE_SIZE, 2 * GAP + 4 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 2 * SQUARE_SIZE, 2 * GAP + 8 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + 3 * SQUARE_SIZE, 2 * GAP + 4 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 3 * SQUARE_SIZE, 2 * GAP + 8 * SQUARE_SIZE), LINE_WIDTH )
        
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, 3 * GAP + 9 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, 3 * GAP + 9 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, 3 * GAP + 10 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, 3 * GAP + 10 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, 3 * GAP + 11 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, 3 * GAP + 11 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + 1 * SQUARE_SIZE, 3 * GAP + 8 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 1 * SQUARE_SIZE, 3 * GAP + 12 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + 2 * SQUARE_SIZE, 3 * GAP + 8 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 2 * SQUARE_SIZE, 3 * GAP + 12 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + 3 * SQUARE_SIZE, 3 * GAP + 8 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 3 * SQUARE_SIZE, 3 * GAP + 12 * SQUARE_SIZE), LINE_WIDTH )
        
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, 4 * GAP + 13 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, 4 * GAP + 13 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, 4 * GAP + 14 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, 4 * GAP + 14 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP, 4 * GAP + 15 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 4 * SQUARE_SIZE, 4 * GAP + 15 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + 1 * SQUARE_SIZE, 4 * GAP + 12 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 1 * SQUARE_SIZE, 4 * GAP + 16 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + 2 * SQUARE_SIZE, 4 * GAP + 12 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 2 * SQUARE_SIZE, 4 * GAP + 16 * SQUARE_SIZE), LINE_WIDTH )
        pygame.draw.line( self.screen, LINE_COLOR, (GAP + TEXT_LENGTH + GAP + 3 * SQUARE_SIZE, 4 * GAP + 12 * SQUARE_SIZE), (GAP + TEXT_LENGTH + GAP + 3 * SQUARE_SIZE, 4 * GAP + 16 * SQUARE_SIZE), LINE_WIDTH )

        font = pygame.font.Font("Daydream.ttf", 32)
        
        floor4_text = font.render('FLOOR 4', True, CIRCLE_COLOR, BG_COLOR)
        floor4_textRect = floor4_text.get_rect()
        floor4_textRect.center = (180, 180)
        self.screen.blit(floor4_text, floor4_textRect)
        
        floor3_text = font.render('FLOOR 3', True, CIRCLE_COLOR, BG_COLOR)
        floor3_textRect = floor3_text.get_rect()
        floor3_textRect.center = (180, 180 + 1 * GAP + 4 * SQUARE_SIZE)
        self.screen.blit(floor3_text, floor3_textRect)

        floor2_text = font.render('FLOOR 2', True, CIRCLE_COLOR, BG_COLOR)
        floor2_textRect = floor2_text.get_rect()
        floor2_textRect.center = (180, 180 + 2 * GAP + 8 * SQUARE_SIZE)
        self.screen.blit(floor2_text, floor2_textRect)

        floor1_text = font.render('FLOOR 1', True, CIRCLE_COLOR, BG_COLOR)
        floor1_textRect = floor1_text.get_rect()
        floor1_textRect.center = (180, 180 + 3 * GAP + 12 * SQUARE_SIZE)
        self.screen.blit(floor1_text, floor1_textRect)

        pygame.display.update()
        self.clock.tick(self.fps)

    def draw_figures(self):
        if self.screen is None:
            return
        for floor in range(BOARD_FLOORS):
            for row in range(BOARD_ROWS):
                for col in range(BOARD_COLS):
                    if self.board[3-floor, row, col] == 1:
                        pygame.draw.circle(
                            self.screen,
                            CIRCLE_COLOR,
                            (
                                int( GAP + TEXT_LENGTH + GAP + col * SQUARE_SIZE + SQUARE_SIZE//2 ),
                                int( GAP + floor*(GAP + 4*SQUARE_SIZE) + row * SQUARE_SIZE + SQUARE_SIZE//2 )
                            ),
                            CIRCLE_RADIUS,
                            CIRCLE_WIDTH
                        )
                    elif self.board[3-floor, row, col] == -1:
                        pygame.draw.line(
                            self.screen,
                            CROSS_COLOR,
                            (
                                GAP + TEXT_LENGTH + GAP + col * SQUARE_SIZE + SPACE,
                                GAP + floor*(GAP + 4*SQUARE_SIZE) + row * SQUARE_SIZE + SQUARE_SIZE - SPACE
                            ),
                            (
                                GAP + TEXT_LENGTH + GAP + col * SQUARE_SIZE + SQUARE_SIZE - SPACE,
                                GAP + floor*(GAP + 4*SQUARE_SIZE) + row * SQUARE_SIZE + SPACE
                            ),
                            CROSS_WIDTH
                        )	
                        pygame.draw.line(
                            self.screen,
                            CROSS_COLOR,
                            (
                                GAP + TEXT_LENGTH + GAP + col * SQUARE_SIZE + SPACE,
                                GAP + floor*(GAP + 4*SQUARE_SIZE) + row * SQUARE_SIZE + SPACE
                            ),
                            (
                                GAP + TEXT_LENGTH + GAP + col * SQUARE_SIZE + SQUARE_SIZE - SPACE,
                                GAP + floor*(GAP + 4*SQUARE_SIZE) + row * SQUARE_SIZE + SQUARE_SIZE - SPACE
                            ),
                            CROSS_WIDTH
                        )
        pygame.display.update()
        self.clock.tick(self.fps)

In [4]:
# game = TicTacToe3D(headless=False)
# print(game.all_wins[0])